In [1]:
import geopandas as gpd
import pandas as pd
import os

# Explore geology as covariate

According to Bogen (1996), SS in the Atna at Li Bru (`2.479.0`) comes from a small area of Pleistocene sediments (~30 km2), with the rest of the catchment (>110 km2) generating essentially nothing. This notebook uses geological map data from NGU to summarise geology in this catchment. 

Unfortunately, the NGU bedrock dataset does not show any "løsmasse" in this area. By contrast, the Løsmasse kart shows løsmasse everywhere, and it is not clear how to estimate anything comparable to Bogen's 30 km2 of Pleistocene sediments.

In [2]:
dir_path = r"/home/jovyan/shared/common/teotil3/nve_ss_data"
cat_gdf = gpd.read_file(os.path.join(dir_path, "filtered_catchments.gpkg")).query("station_id == '2.479.0'")
geo_gdf = gpd.read_file(os.path.join(dir_path, "BerggrunnN50.gdb"), layer="BergartFlate_N50")

print("Reprojecting to equal area...")
cat_gdf_cea = cat_gdf.to_crs({"proj": "cea"})
geo_gdf_cea = geo_gdf.to_crs({"proj": "cea"})
cat_gdf_cea.sindex
geo_gdf_cea.sindex

print("Intersecting polygons...")
int_gdf = gpd.overlay(
    cat_gdf_cea, geo_gdf_cea[['hovedbergart', 'geometry']], how="intersection", keep_geom_type=True
)
int_gdf["area_km2"] = int_gdf["geometry"].area / 1e6

print("Aggregating...")
lc_df = int_gdf.groupby(["station_id", "hovedbergart"]).sum(numeric_only=True)["area_km2"]
lc_df = lc_df.unstack("hovedbergart")
lc_df.columns = [f"a_{i}_km2" for i in lc_df.columns]
lc_df.reset_index(inplace=True)
lc_df.columns.name = ""

# cat_gdf = pd.merge(cat_gdf, lc_df, on="station_id", how="left")
# cat_gdf['a_løsmasser_km2'] = cat_gdf['a_løsmasser_km2'].fillna(0)
lc_df

/opt/conda/lib/python3.11/site-packages/pyogrio/raw.py:194: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  result = ogr_read(
/opt/conda/lib/python3.11/site-packages/pyogrio/geopandas.py:49: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  res = pd.to_datetime(ser, **datetime_kwargs)
/opt/conda/lib/python3.11/site-packages/pyogrio/geopandas.py:49: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  res = pd.to_datetime(ser, **datetime_kwargs)


Reprojecting to equal area...
Intersecting polygons...
Aggregating...


,station_id,a_304.0_km2,a_306.0_km2,a_321.0_km2,a_412.0_km2,a_424.0_km2
0,2.479.0,112.909596,24.105067,0.063953,16.464968,2.940982
